In [10]:
import random
import numpy as np
import bs4 as BS
import requests as rqs
import pandas as pd 

In [2]:
pages = random.sample(range(1, 193),192)
pages.sort()

In [3]:
year = []
car = []
price = []
for page in pages:

    source = rqs.get(r'https://www.cheki.co.ke/vehicles/toyota?page={}'.format(page)).text

    soup = BS.BeautifulSoup(source, 'lxml')

    for article in soup.find_all('div',class_= 'right-card-part-container'):

        year.append(article.find('span', class_='ellipses').text.split('\n')[2])

        car.append(article.find('span', class_='ellipses').text.split('\n')[3])
        try:

            price.append(article.find('h2', class_= 'listing-price').text)
        except AttributeError:
            price.append('0')

In [8]:
Dict = {
    'Car': car,
    'Year': year,
    'Price': price
}

In [12]:
df = pd.DataFrame(Dict)

In [13]:
df.head()

,Car,Year,Price
0,Toyota Voxy,2013,"\nKSh1,475,000\n"
1,Toyota Spade,2013,"\nKSh750,000\n"
2,Toyota Hilux 2.5 D-4D Double Cab,2013,"\nKSh2,400,000\n"
3,Toyota Ractis,2013,"\nKSh880,000\n"
4,Toyota Passo,2013,"\nKSh635,000\n"


In [25]:
df['Price'] = pd.to_numeric(df['Price'].apply(lambda x: x.replace('\n', '').replace('KSh', '').replace(',','')))

In [27]:
df.head(2)

,Car,Year,Price
0,Toyota Voxy,2013,1475000
1,Toyota Spade,2013,750000


In [31]:
filt = (df['Year'] == 'Toyota Vanguard')

In [34]:
incorrect = df[filt].index

In [37]:
df = df.drop(incorrect).reset_index(drop = True)

In [39]:
df['Year'] = pd.to_numeric(df['Year'])

In [41]:
df = df.rename(columns = {'Price': 'Price in KSh'})

In [42]:
df.head(2)

,Car,Year,Price in KSh
0,Toyota Voxy,2013,1475000
1,Toyota Spade,2013,750000


In [53]:
car = df.groupby('Car')

In [56]:
car.get_group('Toyota Spade')

,Car,Year,Price in KSh
1,Toyota Spade,2013,750000
23,Toyota Spade,2013,750000
45,Toyota Spade,2013,750000
507,Toyota Spade,2013,750000
704,Toyota Spade,2013,835000
1359,Toyota Spade,2012,900000
1544,Toyota Spade,2012,850000
2053,Toyota Spade,2012,740000


In [63]:
pd.set_option('display.max_rows',1000)

In [69]:
df[df['Car'].str.contains('HiAce')].sort_values(by = 'Price in KSh').reset_index(drop = True)

,Car,Year,Price in KSh
0,Toyota HiAce,1988,465000
1,Toyota HiAce,1999,635000
2,Toyota HiAce,1999,720000
3,Toyota HiAce,2006,900000
4,Toyota HiAce,2007,1350000
5,Toyota HiAce,2009,1350000
6,Toyota HiAce,2006,1350000
7,Toyota HiAce KDH200,2008,1490000
8,Toyota HiAce,2008,1500000
9,Toyota HiAce KDH200,2008,1500000


In [1]:
import string